# Datos geográficos del gobierno boliviano

> Inventario de datos geográficos publicados por entidades de gobierno en Bolivia

In [1]:
import xmltodict
import requests
import datetime as dt
from requests.adapters import HTTPAdapter
import pandas as pd
from IPython.display import display, Markdown
from itables import show, init_notebook_mode
init_notebook_mode()

pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

<IPython.core.display.Javascript object>

In [2]:
def create_session(retries):
    s = requests.Session()
    s.mount('http://', HTTPAdapter(max_retries=retries))
    s.mount('https://', HTTPAdapter(max_retries=retries))
    return s

def get_capas(geoserver_domain, geoserver_name, decode=True):
    columns = ['Name', 'Title', 'Abstract', 'CRS', 'EX_GeographicBoundingBox.westBoundLongitude', 'EX_GeographicBoundingBox.eastBoundLongitude', 'EX_GeographicBoundingBox.southBoundLatitude', 'EX_GeographicBoundingBox.northBoundLatitude', 'KeywordList.Keyword']
    column_names = ['nombre', 'titulo', 'abstract', 'crs', 'limite_oeste', 'limite_este', 'limite_sur', 'limite_norte', 'keywords']
    url = 'http://{}/geoserver/ows?service=wms&request=GetCapabilities'.format(geoserver_domain)
    print(url)
    try:
        response = s.get(url, timeout=20)
        capabilities = xmltodict.parse(response.text)
        capas = pd.json_normalize(capabilities['WMS_Capabilities']['Capability']['Layer']['Layer'])
        print(len(capas))
        capas = capas[columns]
        capas.columns = column_names
        capas['geojson'] = capas.nombre.apply(lambda nombre: 'http://{}/geoserver/ows?service=WFS&request=GetFeature&typeName={}&outputFormat={}'.format(geoserver_domain, nombre, 'application/json'))
        capas['csv'] = capas.nombre.apply(lambda nombre: 'http://{}/geoserver/ows?service=WFS&request=GetFeature&typeName={}&outputFormat={}'.format(geoserver_domain, nombre, 'csv'))
        capas['keywords'] = capas['keywords'].fillna('').apply(lambda keys: [k for k in keys if type(k) == str] if type(keys) == list else keys)
        capas.insert(0, 'sistema', geoserver_name)
        if decode:
            for column in ['nombre', 'titulo', 'abstract']:
                capas[column] = capas[column].apply(lambda x: x.encode("iso-8859-1").decode('utf-8') if type(x) == str else x)
            capas['keywords'] = capas['keywords'].apply(lambda keys: [k.encode("iso-8859-1").decode('utf-8') for k in keys])
        return capas
    except Exception as e:
        print(e)
        return pd.DataFrame()

def update(geoserver_domains):
    geodata = []
    for entry in geoserver_domains:
        capas = get_capas(geoserver_domain=entry['domain'], geoserver_name=entry['short_name'], decode=entry['decode'])
        geodata.append(capas)
    return pd.concat(geodata)

geoserver_domains = [
    dict(domain='geoserver.oopp.gob.bo', decode=True, name='Ministerio de Obras Públicas, Servicios y Vivienda', short_name='MOPSV'), 
    dict(domain='geo.gob.bo', decode=False, name='GeoBolivia', short_name='GeoBolivia'), 
    dict(domain='sig01.mmaya.gob.bo', decode=True, name='Ministerio de Medio Ambiente y Agua', short_name='MMAYA'),
    dict(domain='siip.produccion.gob.bo:8080', decode=True, name='Sistema Integrado de Información Productiva', short_name='SIIP'),
    dict(domain='sitservicios.lapaz.bo', decode=True, name='Sistema de Información Territorial del Gobierno Autónomo Municipal de La Paz', short_name='SIT La Paz'),
    dict(domain='geosunit.vicetierras.gob.bo', decode=True, name='GeoSUNIT del Viceministerio de Tierras', short_name='GeoSunit'),
    dict(domain='incendios.fan-bo.org:8080', decode=True, name='Sistema de monitoreo y alerta temprana de riesgos de incendios forestales', short_name='FAN'),
    dict(domain='seie.minedu.gob.bo:8080', decode=True, name='Sistema de Estadísticas e Indicadores Educativos', short_name='SEIE'),
    # dict(domain='geo03.siarh.gob.bo', decode=True, name='GeoSIARH Sistema de Información Ambiental y de Recursos Hídricos', short_name='GeoSIARH'),
    # dict(domain='geonode.fonadin.gob.bo', decode=True, name='Fondo Nacional de Desarrollo Integral', short_name='FONADIN'),
    dict(domain='sigvmeea.hidrocarburos.gob.bo', decode=True, name='Sistema de Información Geográfica en Línea para Electricidad y Energías Alternativas', short_name='Sigvmeea'),
    # dict(domain='geoportal.ypfb.gob.bo', decode=True, name='Infraestructura de Datos Espaciales de Hidrocarburos', short_name='YPFB')
]

s = create_session(5)
data = update(geoserver_domains)
data.to_csv('geodatos.csv', index=False)

http://geoserver.oopp.gob.bo/geoserver/ows?service=wms&request=GetCapabilities
51
http://geo.gob.bo/geoserver/ows?service=wms&request=GetCapabilities
HTTPSConnectionPool(host='geo.gob.bo', port=443): Read timed out.
http://sig01.mmaya.gob.bo/geoserver/ows?service=wms&request=GetCapabilities
220
http://siip.produccion.gob.bo:8080/geoserver/ows?service=wms&request=GetCapabilities
81
http://sitservicios.lapaz.bo/geoserver/ows?service=wms&request=GetCapabilities
266
http://geosunit.vicetierras.gob.bo/geoserver/ows?service=wms&request=GetCapabilities
51
http://incendios.fan-bo.org:8080/geoserver/ows?service=wms&request=GetCapabilities
96
http://seie.minedu.gob.bo:8080/geoserver/ows?service=wms&request=GetCapabilities
26
http://sigvmeea.hidrocarburos.gob.bo/geoserver/ows?service=wms&request=GetCapabilities
194


In [3]:
def resumen_sistemas():
    sistemas = {g['short_name']:g['name'] for g in geoserver_domains}
    dfi = data.sistema.value_counts().reset_index()
    dfi.columns = ['Abreviación', 'Número de capas']
    dfi.insert(1, 'Sistema', dfi.Abreviación.map(sistemas))
    return dfi

display(Markdown('**Número de capas por sistema geográfico consultado:**'))
display(resumen_sistemas())

**Número de capas por sistema geográfico consultado:**

,Abreviación,Sistema,Número de capas
0,SIT La Paz,Sistema de Información Territorial del Gobierno Autónomo Municipal de La Paz,266
1,MMAYA,Ministerio de Medio Ambiente y Agua,220
2,Sigvmeea,Sistema de Información Geográfica en Línea para Electricidad y Energías Alternativas,194
3,FAN,Sistema de monitoreo y alerta temprana de riesgos de incendios forestales,96
4,SIIP,Sistema Integrado de Información Productiva,81
5,GeoSunit,GeoSUNIT del Viceministerio de Tierras,51
6,MOPSV,"Ministerio de Obras Públicas, Servicios y Vivienda",51
7,SEIE,Sistema de Estadísticas e Indicadores Educativos,26


In [4]:
def draw_table(df):
    
    dfi = df[['sistema', 'titulo', 'abstract', 'geojson', 'csv']].copy()
    dfi = dfi.drop_duplicates(subset=['titulo'])
    dfi['titulo'] = dfi['titulo'].apply(lambda n: '<b>{}</b>'.format(n))
    dfi['geojson'] = dfi['geojson'].apply(lambda url: '<a href="{}">{}</a>'.format(url, "geojson"))
    dfi['csv'] = dfi['csv'].apply(lambda url: '<a href="{}">{}</a>'.format(url, "csv"))
    dfi['abstract'] = dfi['abstract'].fillna('')
    # dfi['keywords'] = dfi['keywords'].fillna('').apply(lambda keys: ', '.join([str(k) for k in keys if k not in ['features']]) if type(keys) == list else str(keys))
    dfi.columns = ['Sistema', 'Nombre', 'Descripción', '', ' ']
    dfi = dfi.set_index('Nombre').sort_index(ascending=True)

    show(dfi, 
         order = [],
         hover = True,
         compact=True,
         scrollY="900px", 
         lengthMenu=[50,100],
         scrollCollapse=True,
         search={"caseInsensitive": True},
         paging=True,
         language={
             'lengthMenu': 'Mostrar _MENU_ filas',
             'search': '&#x1F50E;&#xFE0E;', 
             'processing': 'creando tabla ...', 
             'info': '', 
             'infoEmpty': '', 
             'infoFiltered':'_TOTAL_ documentos',
             'paginate': {
                'first': "Primero",
                'previous': "Anterior",
                'next': "Siguiente",
                'last': "Último"
            },
         }, 
         maxBytes=0,
         columnDefs=[
             {"width": "5px", "targets": [3, 4]},
             {"width": "20px", "targets": [0, 1]},
             {"width": "100px", "targets": 2}
         ]
        )

draw_table(data)

Loading... (need help?)


In [5]:
display(Markdown("- Datos actualizados en {}.\n- [Más información sobre cada capa](https://flatgithub.com/mauforonda/geodatos?filename=geodatos.csv&filters=&sort=titulo%2Casc&stickyColumnName=titulo).\n- [Historial de cambios](https://github.com/mauforonda/geodatos/commits/master/geodatos.csv).\n- Nuevos servicios y datos serán probablemente agregados en el futuro.".format(dt.datetime.now().strftime('**%Y-%m-%d**'))))

- Datos actualizados en **2022-03-05**.
- [Más información sobre cada capa](https://flatgithub.com/mauforonda/geodatos?filename=geodatos.csv&filters=&sort=titulo%2Casc&stickyColumnName=titulo).
- [Historial de cambios](https://github.com/mauforonda/geodatos/commits/master/geodatos.csv).
- Nuevos servicios y datos serán probablemente agregados en el futuro.